In [2]:
# David Avallone & Kelly Reynolds Computer Vision

import cv2 
from matplotlib import pyplot as plt
import math
import numpy as np
import os
from PIL import Image

In [61]:
centroid = (0,0)
# Holds the iris' radius
radius = 0
# Holds the current element of the image used by the getNewEye function
currentEye = 0
# Holds the list of eyes (filenames)
eyesList = []

def getNewEye(list):
	global currentEye
	if (currentEye >= len(list)):
		currentEye = 0
	newEye = list[currentEye]
	currentEye += 1
	return (newEye)

def getCircles(image):
    i = 80
    while i < 151:
        circles = cv2.HoughCircles(image, cv2.HOUGH_GRADIENT, dp=2, minDist=105.0, param1=30, param2=i, minRadius=100, maxRadius=140)
        if circles is not None and len(circles) == 1:
            return circles[0]
        i += 1
    return []

def getPupil(frame):
    # Convert frame to UMat
    frame = cv2.UMat(frame)
    # Thresholding
    pupilImg = cv2.inRange(frame, (30,30,30), (80,80,80))
    # Find contours
    contours, _ = cv2.findContours(pupilImg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Convert UMat to numpy array for copying
    frame = cv2.UMat.get(frame)
    # Draw circle around pupil
    pupilImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    pupilImg = cv2.cvtColor(pupilImg, cv2.COLOR_GRAY2BGR)
    for contour in contours:
        moments = cv2.moments(contour)
        area = moments['m00']
        if (area > 50):
            x = int(moments['m10'] / area)
            y = int(moments['m01'] / area)
            radius = int(np.sqrt(area / np.pi))
            cv2.circle(pupilImg, (x, y), radius, (0, 0, 0), cv2.FILLED)
            break

    return pupilImg

def getIris1(frame):
    copyImg = frame.copy()
    resImg = frame.copy()
    grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(frame)
    grayImg = cv2.Canny(grayImg, 5, 70)
    grayImg = cv2.GaussianBlur(grayImg, (7, 7), 0)
    circles = cv2.HoughCircles(grayImg, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=50, param2=50, minRadius=0, maxRadius=0)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            rad = int(circle[2])
            global centroid
            centroid = (int(circle[0]), int(circle[1]))
            cv2.circle(mask, centroid, rad, (255, 255, 255), -1)
            mask = cv2.bitwise_not(mask)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)  # Convert mask to single-channel
            cv2.subtract(frame, copyImg, resImg, mask)
            cv2.circle(resImg, centroid, rad, (0, 0, 0), 2)
            break

    return resImg

def getIris2(frame):
    copyImg = frame.copy()
    resImg = frame.copy()
    grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(frame)
    grayImg = cv2.Canny(grayImg, 5, 70)
    grayImg = cv2.GaussianBlur(grayImg, (7, 7), 0)
    circles = getCircles(grayImg)
    if len(circles) > 0:
        for circle in circles:
            rad = int(circle[2])
            global centroid
            centroid = (int(circle[0]), int(circle[1]))
            cv2.circle(mask, centroid, rad, (255, 255, 255), -1)
            mask = cv2.bitwise_not(mask)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)  # Convert mask to single-channel
            cv2.subtract(frame, copyImg, resImg, mask)
            cv2.circle(resImg, centroid, rad, (0, 0, 0), 2)
            break

    return resImg

def process_eyes(directory, output_directory):
    # Get a list of all subdirectories (001, 002, etc.) in the directory
    subdirectories = [f.path for f in os.scandir(directory) if f.is_dir()]
    
    for subdir in subdirectories:
        # Create a new directory for the processed images
        new_subdir = os.path.join(output_directory, os.path.basename(subdir))
        os.makedirs(new_subdir, exist_ok=True)

        # Process images in both 'L' and 'R' subdirectories
        for sub_eye_dir in ['L', 'R']:
            eyesList = os.listdir(os.path.join(subdir, sub_eye_dir))

            for eye in eyesList:
                image_path = os.path.join(subdir, sub_eye_dir, eye)
                if not os.path.isfile(image_path):
                    print(f"File not found: {image_path}")
                    continue

                frame = cv2.imread(image_path)
                if frame is None:
                    print(f"Error loading image: {image_path}")
                    continue

                # Process the image to get the iris
                output = getPupil(frame)
                iris = getIris2(output)

                # Save the processed iris image
                save_path = os.path.join(new_subdir, f"{sub_eye_dir}_iris_{eye}")
                cv2.imwrite(save_path, iris)

def process_eyes1(directory, output_directory):
    # Get a list of all subdirectories (001, 002, etc.) in the directory
    subdirectories = [f.path for f in os.scandir(directory) if f.is_dir()]

    for subdir in subdirectories:
        # Create a new directory for the processed images
        new_subdir = os.path.join(output_directory, os.path.basename(subdir))
        os.makedirs(new_subdir, exist_ok=True)

        # Process images in the subdirectory
        eyesList = os.listdir(subdir)

        for eye in eyesList:
            if not eye.startswith('L_iris_') and not eye.startswith('R_iris_'):
                continue
            
            image_path = os.path.join(subdir, eye)
            if not os.path.isfile(image_path):
                print(f"File not found: {image_path}")
                continue

            frame = cv2.imread(image_path)
            if frame is None:
                print(f"Error loading image: {image_path}")
                continue

            # Process the image to get the iris
            output = getPupil(frame)
            iris = getIris1(output)

            # Save the processed iris image
            save_path = os.path.join(new_subdir, eye)
            cv2.imwrite(save_path, iris)

In [41]:
def crop_image(image_path, new_image_name, crop_coordinates):
    with Image.open(image_path) as img:
        # Crop the image
        cropped_img = img.crop(crop_coordinates)
        # Save the cropped image with the new name
        cropped_img.save(new_image_name)

def crop_images_in_subdirectories(directory,output_directory, crop_coordinates):
    # Get a list of all subdirectories (001, 002, etc.) in the directory
    subdirectories = [f.path for f in os.scandir(directory) if f.is_dir()]
    
    for subdir in subdirectories:
        # Create a new directory for the processed images
        new_subdir = os.path.join(output_directory, os.path.basename(subdir))
        os.makedirs(new_subdir, exist_ok=True)

        # Process images in both 'L' and 'R' subdirectories
        for sub_eye_dir in ['L', 'R']:
            eyesList = os.listdir(os.path.join(subdir, sub_eye_dir))

            for eye in eyesList:
                image_path = os.path.join(subdir, sub_eye_dir, eye)
                if not os.path.isfile(image_path):
                    print(f"File not found: {image_path}")
                    continue

                frame = cv2.imread(image_path)
                if frame is None:
                    print(f"Error loading image: {image_path}")
                    continue
                
                # Save the processed iris image
                save_path = os.path.join(new_subdir, f"{sub_eye_dir}_iris_{eye}")
                crop_image(image_path, save_path, crop_coordinates)

In [ ]:
# Path to the directory containing the CASIA images
input_directory = 'CASIA-Iris-Lamp/'

# Path to the directory where you want to save the processed images
output_directory = 'Processed-CASIA-Iris-Lamp2/'

# Process the eyes in the input directory and save the processed images in the output directory
process_eyes(input_directory, output_directory)

In [4]:
with Image.open('Processed-CASIA-Iris-Interval/001/L_iris_S1001L01.jpg') as img:
    # Get the size of the image
    width, height = img.size
    print(f"Image size: {width} x {height}")

Image size: 320 x 280


In [10]:
with Image.open('Processed-CASIA-Iris-Lamp/003/L_iris_S2003L02.jpg') as img:
    # Get the size of the image
    width, height = img.size
    print(f"Image size: {width} x {height}")

Image size: 640 x 480


In [55]:
image_path = 'CASIA-Iris-Lamp/001/L/S2001L01.jpg'
new_image_name = 'cropped_image.jpg'
crop_coordinates = (200, 100, 500, 400)  # Example coordinates, adjust as needed

crop_image(image_path, new_image_name, crop_coordinates)

In [56]:
crop_coordinates = (200, 100, 500, 400)  # Example coordinates, adjust as needed

crop_images_in_subdirectories("CASIA-Iris-Lamp","Cropped-CASIA-Iris-Lamp", crop_coordinates)

KeyboardInterrupt: 

In [62]:
process_eyes1("Cropped-CASIA-Iris-Lamp", "Processed-CASIA-Iris-Lamp-Cropped")

KeyboardInterrupt: 